In [1]:
 !pip install nltk

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import re
from keras.models import Sequential
from keras.layers import Input, Embedding,TimeDistributed, LSTM, Dropout, Bidirectional, Dense 
import random
from nltk.corpus import wordnet 
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.utils import plot_model, to_categorical
import nltk
nltk.download('punkt')  
from nltk.corpus import wordnet 
from tqdm import tqdm
import nltk
import subprocess
import logging
import spacy



# Download and unzip wordnet
try:
    nltk.data.find('wordnet.zip')
except:
    nltk.download('wordnet', download_dir='/kaggle/working/')
    command = "unzip /kaggle/working/corpora/wordnet.zip -d /kaggle/working/corpora"
    subprocess.run(command.split())
    nltk.data.path.append('/kaggle/working/')

# Now you can import the NLTK resources as usual
from nltk.corpus import wordnet
nltk.download('punkt')


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /kaggle/working/...
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
file_path = "/kaggle/input/dataaaaa/src (1).txt"

# Read data into an array of strings
with open(file_path, "r") as file:
    src = np.array([line.strip() for line in file])

file_path = "/kaggle/input/dataaaaa/top_decoupled (1).txt"

# Read data into an array of strings
with open(file_path, "r") as file:
    top_decoupled = np.array([line.strip() for line in file])

In [6]:
def preprosses(src,top_decoupled):
    word_tag=[]
    for src,trg in  tqdm(zip(src,top_decoupled)):
        out=extract_bio_tags(src,trg)
        word_tag.append(out)
    return word_tag

In [7]:
  nlp = spacy.load("en_core_web_sm")

In [8]:
def extract_bio_tags(src, top_decoupled):
  
    doc = nlp(str(src))
    pos_tags = [token.pos_ for token in doc]

    # Parse the TOP-DECOUPLED structure
    matches = re.findall(r'\(NOT \([^()]+\) \)|\((?:[^()]+)\)|\(NOT \(COMPLEX_TOPPING \((?:.+?) \) \) \)', top_decoupled)
    ind = -1
    if random.random() < 0.5:
        ind = random.randint(0, len(src.split()) - 1)

    # Map words to BIO tags
    bio_tag_map = {}
    for i, match in enumerate(matches):
        match = re.sub(r'\(|\)', '', match)
        match = match.strip()
        words = match.split()  # Split multi-word values

        if words[0] == "NOT":
            if "COMPLEX_TOPPING" in words:
                words.remove("COMPLEX_TOPPING")
                words.remove("NOT")
                indx = words.index("TOPPING")
                quantity = words[1:indx]
                topping = words[indx + 1:]
                for i, word in enumerate(quantity):
                    if i == 0:
                        bio_tag_map[word] = "B-QUANTITY"  # Begin tag
                    else:
                        bio_tag_map[word] = "I-QUANTITY"  # Inside tag
                for i, word in enumerate(topping):
                    if i == 0:
                        bio_tag_map[word] = "B-NOT-TOPPING"  # Begin tag
                    else:
                        bio_tag_map[word] = "I-NOT-TOPPING"  # Inside tag
                continue
            else:
                tag = words[0] + "-" + words[1]
                words = words[2:]
        else:
            tag = words[0]
            words = words[1:]

        for i, word in enumerate(words):
            if i == 0:
                bio_tag_map[word] = f"B-{tag}"  # Begin tag
            else:
                bio_tag_map[word] = f"I-{tag}"  # Inside tag

    src_words = src.split()
    word_tag_pairs = [(word, bio_tag_map.get(word, "O"), pos_tags[idx]) for idx, word in enumerate(src_words)]
    
    if ind != -1:
        word_tag_pairs[ind] = ("<UNK>", word_tag_pairs[ind][1], word_tag_pairs[ind][2])

    return word_tag_pairs



In [9]:
sentences =preprosses(src,top_decoupled)

16400it [02:00, 136.42it/s]


In [10]:

maxlen = max([len(s) for s in sentences])
words = [w[0] for sentence in sentences for w in sentence]
words = list(set(words))
words.append("<pad>")
n_words = len(words)
print('Number of unique words:', n_words)


Number of unique words: 481


In [11]:


tags = [w[1] for sentence in sentences for w in sentence]
tags=list(set(tags))
print(tags)
n_tags = len(tags)
print('Number of unique Tags:', n_tags)

['B-NUMBER', 'B-DRINKTYPE', 'B-NOT-TOPPING', 'I-SIZE', 'I-QUANTITY', 'I-CONTAINERTYPE', 'B-SIZE', 'I-DRINKTYPE', 'B-NOT-STYLE', 'B-STYLE', 'B-TOPPING', 'I-NUMBER', 'I-VOLUME', 'I-NOT-TOPPING', 'O', 'B-VOLUME', 'I-TOPPING', 'I-STYLE', 'B-CONTAINERTYPE', 'I-NOT-STYLE', 'B-QUANTITY']
Number of unique Tags: 21


In [12]:
pos_tags = [w[2] for sentence in sentences for w in sentence]
pos_tags = list(set(pos_tags))
n_pos_tags = len(pos_tags)
print('Number of unique POS Tags:', n_pos_tags)


Number of unique POS Tags: 17


In [13]:
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}
pos2idx = {p: i for i, p in enumerate(pos_tags)}

print(tag2idx)

{'B-NUMBER': 0, 'B-DRINKTYPE': 1, 'B-NOT-TOPPING': 2, 'I-SIZE': 3, 'I-QUANTITY': 4, 'I-CONTAINERTYPE': 5, 'B-SIZE': 6, 'I-DRINKTYPE': 7, 'B-NOT-STYLE': 8, 'B-STYLE': 9, 'B-TOPPING': 10, 'I-NUMBER': 11, 'I-VOLUME': 12, 'I-NOT-TOPPING': 13, 'O': 14, 'B-VOLUME': 15, 'I-TOPPING': 16, 'I-STYLE': 17, 'B-CONTAINERTYPE': 18, 'I-NOT-STYLE': 19, 'B-QUANTITY': 20}


In [14]:
# Prepare input sequences for words, POS tags, and labels
X_words = [[word2idx[w[0]] for w in s] for s in sentences]
X_words = sequence.pad_sequences(maxlen=maxlen + 40, sequences=X_words, padding="post", value=n_words - 1)

X_pos = [[pos2idx[w[2]] for w in s] for s in sentences]
X_pos = sequence.pad_sequences(maxlen=maxlen + 40, sequences=X_pos, padding="post", value=n_pos_tags - 1)

y = [[tag2idx[w[1]] for w in s] for s in sentences]
y = sequence.pad_sequences(maxlen=maxlen + 40, sequences=y, padding="post", value=tag2idx['O'])
y = np.array([to_categorical(i, num_classes=n_tags) for i in y])

print('X_words shape:', X_words.shape, 'X_pos shape:', X_pos.shape, 'y shape:', y.shape)


X_words shape: (16400, 75) X_pos shape: (16400, 75) y shape: (16400, 75, 21)


In [15]:
# Combining X_words and X_pos into a single feature input
X_combined = np.concatenate([
    np.expand_dims(X_words, axis=-1),  # Add dimension for words
    np.expand_dims(X_pos, axis=-1)    # Add dimension for POS tags
], axis=-1)

print('X_combined shape:', X_combined.shape)

X_combined shape: (16400, 75, 2)


In [16]:
class config():
    VOCAB = n_words
    MAX_LEN = maxlen+40
    N_OUPUT = n_tags
    
    
    EMBEDDING_VECTOR_LENGTH = 256
    N_LSTM_CELLS = 150
    RECURRENT_DROPOUT = 0.4
    
    OUTPUT_ACTIVATION = 'softmax'
    
    LOSS = 'categorical_crossentropy'
    OPTIMIZER = 'adam'
    METRICS = ['accuracy']
    
    MAX_EPOCHS = 5
    

In [17]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=7)
filepath = "model.keras"
ckpt = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
rlp = ReduceLROnPlateau(monitor='loss', patience=3, verbose=1)

model = Sequential()
model.add(
    TimeDistributed(
        Dense(256, activation="relu"), input_shape=(config.MAX_LEN, 2)
    )  # Accepts combined word and POS features
)
model.add(
    Bidirectional(
        LSTM(config.N_LSTM_CELLS, return_sequences=True, recurrent_dropout=config.RECURRENT_DROPOUT)
    )
)
model.add(
    TimeDistributed(
        Dense(config.N_OUPUT, activation=config.OUTPUT_ACTIVATION)
    )
)
model.compile(loss=config.LOSS, optimizer=config.OPTIMIZER, metrics=config.METRICS)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [18]:
history = model.fit(x=X_combined, y=y, validation_split=0.1,
    callbacks=[es, ckpt, rlp], epochs=config.MAX_EPOCHS
)

Epoch 1/5
462/462 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step - accuracy: 0.9106 - loss: 0.3808
Epoch 1: loss improved from inf to 0.26518, saving model to model.keras
462/462 ━━━━━━━━━━━━━━━━━━━━ 114s 226ms/step - accuracy: 0.9106 - loss: 0.3806 - val_accuracy: 0.9419 - val_loss: 0.2016 - learning_rate: 0.0010
Epoch 2/5
462/462 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step - accuracy: 0.9456 - loss: 0.1787
Epoch 2: loss improved from 0.26518 to 0.16808, saving model to model.keras
462/462 ━━━━━━━━━━━━━━━━━━━━ 103s 224ms/step - accuracy: 0.9456 - loss: 0.1787 - val_accuracy: 0.9534 - val_loss: 0.1576 - learning_rate: 0.0010
Epoch 3/5
462/462 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step - accuracy: 0.9553 - loss: 0.1442
Epoch 3: loss improved from 0.16808 to 0.13664, saving model to model.keras
462/462 ━━━━━━━━━━━━━━━━━━━━ 103s 223ms/step - accuracy: 0.9553 - loss: 0.1442 - val_accuracy: 0.9593 - val_loss: 0.1346 - learning_rate: 0.0010
Epoch 4/5
462/462 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step - accuracy: 0.9620 - loss: 

In [19]:
dev_data = pd.read_json("/kaggle/input/pizza-dataset/PIZZA_dev.json", lines=True)
src_test=dev_data["dev.SRC"].to_numpy()
top_test=dev_data["dev.TOP"].to_numpy()


In [20]:

def evaluate(test_src, test_top):
    true_len = 0
    correct = 0

    for src, top in zip(test_src, test_top):
        src_words_original = src.split()
        if len(src_words_original) > config.MAX_LEN:
            continue
        
        true_len += 1

        # Replace unknown words with <UNK>
        for word in src_words_original:
            if word not in word2idx:
                pattern = rf"(?<=\s){word}(?=\s)"
                src = re.sub(pattern, "<UNK>", src)
        
        # Prepare indices for words and POS tags
        src_words = src.split()
        src_indices = [word2idx.get(w, word2idx.get("<UNK>", -1)) for w in src_words]
        src_pos_indices = [pos2idx.get(nlp(w)[0].pos_, -1) for w in src_words]

        # Pad to max length
        pad_token = word2idx.get("<pad>")
        src_indices = src_indices + [pad_token] * (config.MAX_LEN - len(src_indices))
        src_pos_indices = src_pos_indices + [n_pos_tags - 1] * (config.MAX_LEN - len(src_pos_indices))

        # Combine word and POS indices
        src_combined = np.concatenate([
            np.expand_dims(src_indices, axis=-1),
            np.expand_dims(src_pos_indices, axis=-1)
        ], axis=-1)
        src_combined = np.expand_dims(src_combined, axis=0)  # Add batch dimension

        # Get predictions from the model
        predictions = model.predict(src_combined)
        predictions = np.argmax(predictions, axis=-1)  # Convert to tag indices

        # Map predictions and gold labels back to tags
        idx2tag = {v: k for k, v in tag2idx.items()}
        predicted_tags = [idx2tag[i] for i in predictions[0]]
        
        # Prepare gold labels for comparison
        gold_tags = extract_bio_tags(src, top)
        gold_tags = [w[1] for w in gold_tags]  # Extract BIO tags

        # Compare predictions to gold labels
        padded_gold_tags = gold_tags + ["O"] * (config.MAX_LEN - len(gold_tags))
        if predicted_tags == padded_gold_tags:
            correct += 1

    print(f"Accuracy: {correct / true_len * 100:.2f}%")


In [21]:
evaluate(src_test,top_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━

In [ ]:

import json

with open("word2idx_model1.json", "w") as file:
    json.dump(word2idx,file)
with open("tag2idx_model1.json", "w") as file:
    json.dump(tag2idx,file)